In [11]:
import xml.etree.ElementTree as ET

def read_content(xml_file: str):

    tree = ET.parse(xml_file)
    root = tree.getroot()

    list_with_all_boxes = []

    for boxes in root.iter('object'):

        filename = root.find('filename').text

        ymin, xmin, ymax, xmax = None, None, None, None

        ymin = int(boxes.find("bndbox/ymin").text)
        xmin = int(boxes.find("bndbox/xmin").text)
        ymax = int(boxes.find("bndbox/ymax").text)
        xmax = int(boxes.find("bndbox/xmax").text)

        list_with_single_boxes = [xmin, ymin, xmax, ymax]
        list_with_all_boxes.append(list_with_single_boxes)

    return filename, list_with_all_boxes

In [12]:
import cv2
import os


folder_Bboxes="ILSVRC2012_bbox_val_v3"
# folder_Bboxes="ILSVRC2012_bbox_val_v3_All"

bbox_FileNames = []
for filename in os.listdir(folder_Bboxes):
    bbox_FileNames.append(filename)
bbox_FileNames.sort()  

In [13]:
import cv2
import os


folder_Images="ImagentDataset"
# last_fileName=""
imagesFileNames = []
for filename in os.listdir(folder_Images):
#     img = cv2.imread(os.path.join(folder_Images,filename))
#     if img is not None:
    imagesFileNames.append(filename)
imagesFileNames.sort()  

In [14]:
import cv2
import os


Hmpfolder_Images="HeatMaps_Folder_CAM"

# last_fileName=""
# images = []
HmpFileNames = []
for filename in os.listdir(Hmpfolder_Images):
#     img = cv2.imread(os.path.join(Hmpfolder_Images,filename))
#     if img is not None:
    HmpFileNames.append(filename)
HmpFileNames.sort()

In [15]:
# import the necessary packages
from collections import namedtuple
import numpy as np
import cv2
# define the `Detection` object
Detection = namedtuple("Detection", ["image_path", "gt", "pred"])

def bb_intersection_over_union(boxA, boxB):
	# determine the (x, y)-coordinates of the intersection rectangle
	xA = max(boxA[0], boxB[0])
	yA = max(boxA[1], boxB[1])
	xB = min(boxA[2], boxB[2])
	yB = min(boxA[3], boxB[3])
	# compute the area of intersection rectangle
	interArea = max(0, xB - xA + 1) * max(0, yB - yA + 1)
	# compute the area of both the prediction and ground-truth
	# rectangles
	boxAArea = (boxA[2] - boxA[0] + 1) * (boxA[3] - boxA[1] + 1)
	boxBArea = (boxB[2] - boxB[0] + 1) * (boxB[3] - boxB[1] + 1)
	# compute the intersection over union by taking the intersection
	# area and dividing it by the sum of prediction + ground-truth
	# areas - the interesection area
	iou = interArea / float(boxAArea + boxBArea - interArea)
	# return the intersection over union value
	return iou

In [16]:
def im2double(im):
	return cv2.normalize(im.astype('float'), None, 0.0, 1.0, cv2.NORM_MINMAX)

In [17]:
import cv2

def get_Box_From_heatMap(heatMapPath):
    # Grayscale then Otsu's threshold
#     image = cv2.imread('HeatMaps_Folder/ILSVRC2012_val_00000001.jpeg')
    image = cv2.imread(heatMapPath)
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    # original 
    # thresh = cv2.threshold(gray, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)[1]
    thresh = cv2.threshold(gray, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)[1]

    # Find contours
    cnts = cv2.findContours(thresh, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    cnts = cnts[0] if len(cnts) == 2 else cnts[1]
    c= max(cnts, key=cv2.contourArea)
    x,y,w,h = cv2.boundingRect(c)
    cv2.rectangle(image, (x, y), (x + w, y + h), (0,255,0), 3) ### original

    heatM_boxes = [x,y,w,h]

    cv2.waitKey()
    bbox_file1 = heatMapPath.replace('HeatMaps_Folder_CAM/','BBoxFiles_Result_CAM/')
    bbox_file = bbox_file1.replace('.JPEG','.txt')
#     print('bbox_file: ', bbox_file)
    f = open(bbox_file, "w")
    for i in range(2):
        f.write(str(x)+ " ")
        f.write(str(y)+ " ")
        f.write(str(w)+ " ")
        f.write(str(h)+ " ")
    f.close()

    # import json
    # #     with open('FIltersRespAll', 'w') as fout:
    # #         json.dump(str(lstOfAllDictionClasses), fout)
    # with open('BBoxFiles_Result/ILSVRC2012_val_00000001.txt', 'w') as fout:
    #     json.dump(str(heatM_boxes), fout)

    # if len(cnts) == 0:
    #     return
    # else:
    #     # based on contour area, get the maximum contour which is the hand
    #     segmented = max(cnts, key=cv2.contourArea)
    #     return (thresholded, segmented)

In [25]:
def compute_IoU_SuperImposed (imageFileName, original_box, htmap_box):
#     print('image path:', imageFileName)
    detection = Detection(imageFileName, original_box, htmap_box)
    # detection = Detection("Val_HetMaps/hummingbird_heatM.jpeg", boxes[0], heatM_boxes)

    image = cv2.imread(detection.image_path)
    #     det = Detection("bboxgenerator/sample_6.jpg.jpg", boxData_formulate, heat_map_rect)
    cv2.rectangle(image, tuple(detection.gt[:2]), 
        tuple(detection.gt[2:]), (0, 255, 0), 2)
    cv2.rectangle(image, tuple(detection.pred[:2]), 
        tuple(detection.pred[2:]), (0, 0, 255), 2)
    iou = bb_intersection_over_union(detection.gt,detection.pred )
    cv2.putText(image, "IoU: {:.4f}".format(iou), (10, 30),
        cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0, 255, 0), 2)
#     print("{}: {:.4f}".format(detection.image_path, iou))

    # imag = im2double(image)*0.2

#     cv2.imshow("Show", image)
    cv2.waitKey(0)

    import matplotlib.image as mpimg
    import matplotlib.pyplot as plt

# #     ### Added for saving heatmap
#     nameextracted = imageFileName.replace('ResultsIoU_SuperImpsed/','ResultsIoU/')  
#     output_heatM = nameextracted.replace('JPEG','')  
    output_heatM = imageFileName.replace('ImagentDataset','ResultsIoU_SuperImpsed/') 
#     output_heatM = 'ResulstIoU_SuperImpsed/' + imageFileName
    cv2.imwrite(output_heatM, image)
    img_hM=mpimg.imread(output_heatM)
    #plt.imshow(img_hM)
    #return iou

In [19]:
def compute_IoU (imageFileName, original_box, htmap_box):
    detection = Detection(imageFileName, original_box, htmap_box)

    # detection = Detection("Val_HetMaps/hummingbird_heatM.jpeg", boxes[0], heatM_boxes)
#     print('image IoU path', detection.image_path)
    image = cv2.imread(detection.image_path)
    #     det = Detection("bboxgenerator/sample_6.jpg.jpg", boxData_formulate, heat_map_rect)
    cv2.rectangle(image, tuple(detection.gt[:2]), 
        tuple(detection.gt[2:]), (0, 255, 0), 2)
    cv2.rectangle(image, tuple(detection.pred[:2]), 
        tuple(detection.pred[2:]), (0, 0, 255), 2)
    iou = bb_intersection_over_union(detection.gt,detection.pred )
    cv2.putText(image, "IoU: {:.4f}".format(iou), (10, 30),
        cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0, 255, 0), 2)
#     print("{}: {:.4f}".format(detection.image_path, iou))

    # imag = im2double(image)*0.2

#     cv2.imshow("Show", image)
    cv2.waitKey(0)

    import matplotlib.image as mpimg
    import matplotlib.pyplot as plt

    ### Added for saving heatmap
    ### Added for saving heatmap
#     output_heatM = imageFileName.replace('HeatMaps_Folder/','ResultsIoU/')  
#     print('output hmp name: ', output_heatM)
#     cv2.imwrite(output_heatM, image)
#     img_hM=mpimg.imread(output_heatM)
#     plt.imshow(img_hM)
    return iou

# Final accuracy results reported in the paper for CAM are found at the end of final output cel. 

In [21]:
import os
import numpy as np
import cv2
from py_map2jpg import py_map2jpg

imageCount = 0
IoUCorrect = 0
htmpCounted =0
# for image in imagesFileNames: ## original

for image in HmpFileNames: ## modied as some heamaps not generated yet    
#     name, boxes = read_content("ILSVRC2012_bbox_val_v3_IoU"+ "/" + bbox_FileNames[imageCount])
    boxName = image.replace('.JPEG','.xml')
    name, boxes = read_content("ILSVRC2012_bbox_val_v3"+ "/" + boxName)
#      print('box lengt', boxes)
#     print('box lengt', len(boxes))
    if len(boxes)==1: ### ommit images with multiple objects
        box = boxes[0]
        Detection = namedtuple("Detection", ["image_path", "gt", "pred"])
        
#         print('origin box img', bbox_FileNames[imageCount])
#         print('origin hmp img', HmpFileNames[imageCount])
        
        bbox_threshold = [20, 100, 110] # parameters for the bbox generator
        curParaThreshold = str(bbox_threshold[0])+' '+str(bbox_threshold[1])+' '+str(bbox_threshold[2])+' '
        curHeatMapFile = Hmpfolder_Images+ '/' + HmpFileNames[imageCount];
        curImgFile = folder_Images+ '/' + image;      
        #print('hmp name', curHeatMapFile)
        get_Box_From_heatMap(curHeatMapFile)#### get the bbox from heatmap
    #     imageName = imagesFileNames[imageCount]### original
        imageName = HmpFileNames[imageCount] ## modified to depend on HeatMaps_Folder/atmap data not images

       # print('image name:', imageName)
        curBBoxFile = ""
        bboxFileName =""
        if '.JPEG' in imageName:
            #print('bbox name', bboxFileName)
            bboxFileName = imageName.replace('.JPEG','')
            curBBoxFile = 'BBoxFiles_Result_CAM' + '/' + bboxFileName + '.txt';

        # os.system("bboxgenerator/./dt_box "+curHeatMapFile+' '+curParaThreshold+' '+curBBoxFile)


        with open(curBBoxFile) as f:
            for line in f:
                items = [int(x) for x in line.strip().split()]
        boxData1 = np.array(items[0::4]).T
        boxData2 = np.array(items[1::4]).T
        boxData3 = np.array(items[2::4]).T
        boxData4 = np.array(items[3::4]).T

        #print(boxData1)


        boxData_formulate = np.array([boxData1, boxData2, boxData1+boxData3, boxData2+boxData4]).T

        col1 = np.min(np.array([boxData_formulate[:,0], boxData_formulate[:,2]]), axis=0)
        col2 = np.min(np.array([boxData_formulate[:,1], boxData_formulate[:,3]]), axis=0)
        col3 = np.max(np.array([boxData_formulate[:,0], boxData_formulate[:,2]]), axis=0)
        col4 = np.max(np.array([boxData_formulate[:,1], boxData_formulate[:,3]]), axis=0)


        # boxData_formulate = np.array([col1, col2, col3, col4]).T
        boxData_formulate = np.array([col1, col2, col3, col4]).T

        curHeatMap = cv2.imread(curHeatMapFile)
        curImg = cv2.imread(curImgFile)

        # curHeatMap = im2double(curHeatMap)
        # curHeatMap = py_map2jpg(curHeatMap, None, 'jet')
        curHeatMap = im2double(curImg)*0.2+im2double(curHeatMap)*0.7
        origin_box = []
        htmap_box=[]
        for i in range(boxData_formulate.shape[0]): # for each bbox
            if i==0:
                origin_box= [boxData_formulate[i][0], boxData_formulate[i][1], boxData_formulate[i][2], boxData_formulate[i][3]]
#                 print('origin_bbox', origin_box)
#                 print(boxData_formulate[i][:2])
#                 print(boxData_formulate[i][2:])
                cv2.rectangle(curHeatMap, tuple(boxData_formulate[i][:2]), tuple(boxData_formulate[i][2:]), (0,255,0), 3)        
            else:
                htmap_box= [boxData_formulate[i][0], boxData_formulate[i][1], boxData_formulate[i][2], boxData_formulate[i][3]]
#                 print('htmap_box', htmap_box)
#                print(boxData_formulate[i][:2])
#                print(boxData_formulate[i][2:])
                cv2.rectangle(curHeatMap, tuple(boxData_formulate[i][:2]), tuple(boxData_formulate[i][2:]), (255,0,0), 3)
    #         cv2.imshow('bbox', curHeatMap)
    #         cv2.waitKey(0)
        IoU = compute_IoU(curHeatMapFile, box, htmap_box)
#         compute_IoU_SuperImposed(curImgFile, box, htmap_box)
        #print ('IoU', IoU)
        if IoU>=0.5:
            IoUCorrect+=1
        htmpCounted +=1
    imageCount +=1
    print('image number', imageCount)
#     print('image id', HmpFileNames[imageCount])
print('total IoU correct', IoUCorrect)
print('total hmp', htmpCounted)
print(' WSOL accuracy:', IoUCorrect/(htmpCounted +1))
print(' WSOL GT-known error:', 1 - (IoUCorrect/(htmpCounted +1)))
print('total hmp counted', htmpCounted)
print('total images counted', imageCount)
print('total correct', IoUCorrect)

image number 1
image number 2
image number 3
image number 4
image number 5
image number 6
image number 7
image number 8
image number 9
image number 10
image number 11
image number 12
image number 13
image number 14
image number 15
image number 16
image number 17
image number 18
image number 19
image number 20
image number 21
image number 22
image number 23
image number 24
image number 25
image number 26
image number 27
image number 28
image number 29
image number 30
image number 31
image number 32
image number 33
image number 34
image number 35
image number 36
image number 37
image number 38
image number 39
image number 40
image number 41
image number 42
image number 43
image number 44
image number 45
image number 46
image number 47
image number 48
image number 49
image number 50
image number 51
image number 52
image number 53
image number 54
image number 55
image number 56
image number 57
image number 58
image number 59
image number 60
image number 61
image number 62
image number 63
i

image number 493
image number 494
image number 495
image number 496
image number 497
image number 498
image number 499
image number 500
image number 501
image number 502
image number 503
image number 504
image number 505
image number 506
image number 507
image number 508
image number 509
image number 510
image number 511
image number 512
image number 513
image number 514
image number 515
image number 516
image number 517
image number 518
image number 519
image number 520
image number 521
image number 522
image number 523
image number 524
image number 525
image number 526
image number 527
image number 528
image number 529
image number 530
image number 531
image number 532
image number 533
image number 534
image number 535
image number 536
image number 537
image number 538
image number 539
image number 540
image number 541
image number 542
image number 543
image number 544
image number 545
image number 546
image number 547
image number 548
image number 549
image number 550
image number 5

image number 977
image number 978
image number 979
image number 980
image number 981
image number 982
image number 983
image number 984
image number 985
image number 986
image number 987
image number 988
image number 989
image number 990
image number 991
image number 992
image number 993
image number 994
image number 995
image number 996
image number 997
image number 998
image number 999
image number 1000
image number 1001
image number 1002
image number 1003
image number 1004
image number 1005
image number 1006
image number 1007
image number 1008
image number 1009
image number 1010
image number 1011
image number 1012
image number 1013
image number 1014
image number 1015
image number 1016
image number 1017
image number 1018
image number 1019
image number 1020
image number 1021
image number 1022
image number 1023
image number 1024
image number 1025
image number 1026
image number 1027
image number 1028
image number 1029
image number 1030
image number 1031
image number 1032
image number 10

image number 1435
image number 1436
image number 1437
image number 1438
image number 1439
image number 1440
image number 1441
image number 1442
image number 1443
image number 1444
image number 1445
image number 1446
image number 1447
image number 1448
image number 1449
image number 1450
image number 1451
image number 1452
image number 1453
image number 1454
image number 1455
image number 1456
image number 1457
image number 1458
image number 1459
image number 1460
image number 1461
image number 1462
image number 1463
image number 1464
image number 1465
image number 1466
image number 1467
image number 1468
image number 1469
image number 1470
image number 1471
image number 1472
image number 1473
image number 1474
image number 1475
image number 1476
image number 1477
image number 1478
image number 1479
image number 1480
image number 1481
image number 1482
image number 1483
image number 1484
image number 1485
image number 1486
image number 1487
image number 1488
image number 1489
image numb

image number 1892
image number 1893
image number 1894
image number 1895
image number 1896
image number 1897
image number 1898
image number 1899
image number 1900
image number 1901
image number 1902
image number 1903
image number 1904
image number 1905
image number 1906
image number 1907
image number 1908
image number 1909
image number 1910
image number 1911
image number 1912
image number 1913
image number 1914
image number 1915
image number 1916
image number 1917
image number 1918
image number 1919
image number 1920
image number 1921
image number 1922
image number 1923
image number 1924
image number 1925
image number 1926
image number 1927
image number 1928
image number 1929
image number 1930
image number 1931
image number 1932
image number 1933
image number 1934
image number 1935
image number 1936
image number 1937
image number 1938
image number 1939
image number 1940
image number 1941
image number 1942
image number 1943
image number 1944
image number 1945
image number 1946
image numb

...

image number 36530
image number 36531
image number 36532
image number 36533
image number 36534
image number 36535
image number 36536
image number 36537
image number 36538
image number 36539
image number 36540
image number 36541
image number 36542
image number 36543
image number 36544
image number 36545
image number 36546
image number 36547
image number 36548
image number 36549
image number 36550
image number 36551
image number 36552
image number 36553
image number 36554
image number 36555
image number 36556
image number 36557
image number 36558
image number 36559
image number 36560
image number 36561
image number 36562
image number 36563
image number 36564
image number 36565
image number 36566
image number 36567
image number 36568
image number 36569
image number 36570
image number 36571
image number 36572
image number 36573
image number 36574
image number 36575
image number 36576
image number 36577
image number 36578
image number 36579
image number 36580
image number 36581
image number

image number 36963
image number 36964
image number 36965
image number 36966
image number 36967
image number 36968
image number 36969
image number 36970
image number 36971
image number 36972
image number 36973
image number 36974
image number 36975
image number 36976
image number 36977
image number 36978
image number 36979
image number 36980
image number 36981
image number 36982
image number 36983
image number 36984
image number 36985
image number 36986
image number 36987
image number 36988
image number 36989
image number 36990
image number 36991
image number 36992
image number 36993
image number 36994
image number 36995
image number 36996
image number 36997
image number 36998
image number 36999
image number 37000
image number 37001
image number 37002
image number 37003
image number 37004
image number 37005
image number 37006
image number 37007
image number 37008
image number 37009
image number 37010
image number 37011
image number 37012
image number 37013
image number 37014
image number

image number 37397
image number 37398
image number 37399
image number 37400
image number 37401
image number 37402
image number 37403
image number 37404
image number 37405
image number 37406
image number 37407
image number 37408
image number 37409
image number 37410
image number 37411
image number 37412
image number 37413
image number 37414
image number 37415
image number 37416
image number 37417
image number 37418
image number 37419
image number 37420
image number 37421
image number 37422
image number 37423
image number 37424
image number 37425
image number 37426
image number 37427
image number 37428
image number 37429
image number 37430
image number 37431
image number 37432
image number 37433
image number 37434
image number 37435
image number 37436
image number 37437
image number 37438
image number 37439
image number 37440
image number 37441
image number 37442
image number 37443
image number 37444
image number 37445
image number 37446
image number 37447
image number 37448
image number

image number 37830
image number 37831
image number 37832
image number 37833
image number 37834
image number 37835
image number 37836
image number 37837
image number 37838
image number 37839
image number 37840
image number 37841
image number 37842
image number 37843
image number 37844
image number 37845
image number 37846
image number 37847
image number 37848
image number 37849
image number 37850
image number 37851
image number 37852
image number 37853
image number 37854
image number 37855
image number 37856
image number 37857
image number 37858
image number 37859
image number 37860
image number 37861
image number 37862
image number 37863
image number 37864
image number 37865
image number 37866
image number 37867
image number 37868
image number 37869
image number 37870
image number 37871
image number 37872
image number 37873
image number 37874
image number 37875
image number 37876
image number 37877
image number 37878
image number 37879
image number 37880
image number 37881
image number

image number 38262
image number 38263
image number 38264
image number 38265
image number 38266
image number 38267
image number 38268
image number 38269
image number 38270
image number 38271
image number 38272
image number 38273
image number 38274
image number 38275
image number 38276
image number 38277
image number 38278
image number 38279
image number 38280
image number 38281
image number 38282
image number 38283
image number 38284
image number 38285
total IoU correct 12142
total hmp 38285
 WSOL accuracy: 0.31713942433265424
 WSOL GT-known error: 0.6828605756673458
total hmp counted 38285
total images counted 38285
total correct 12142
